In [1]:
from datasets import load_dataset
from transformers import AutoTokenizer, TrainingArguments, Trainer, AutoModelForSequenceClassification, BigBirdForSequenceClassification
import torch
from transformers import DataCollatorWithPadding
import evaluate
import numpy as np
from evaluate import evaluator
import torch.cuda

/home/leontioso/anaconda3/envs/msc_thesis/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-03-03 09:33:35.080497: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-03 09:33:35.080543: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-03 09:33:35.082858: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-03 09:33:35.330828: I tensorflow/core/platform/cpu_feature_guard.cc:182] This 

In [2]:
echr = load_dataset("ecthr_cases",  "violation-prediction")

In [3]:
tokenizer = AutoTokenizer.from_pretrained('google/bigbird-roberta-base', use_fast = False)

In [4]:
def encode(examples):
    return tokenizer(examples["text"],
                     truncation=True,
                     padding=True)

In [5]:
train_dataset, val_dataset, test_dataset = echr['train'], echr['validation'], echr['test']
train_dataset, val_dataset, test_dataset = [dataset.map( lambda examples: {"text": "\n".join(examples["facts"])}) for dataset in [train_dataset, val_dataset, test_dataset]]
train_dataset, val_dataset, test_dataset = [dataset.map(encode, batched=True) for dataset in [train_dataset, val_dataset, test_dataset]]
train_dataset, val_dataset, test_dataset = [dataset.map( lambda examples: {'labels' :list(1 if examples['labels'][i] else 0 for i in range(len(examples['labels'])))}, batched=True) for dataset in [train_dataset, val_dataset, test_dataset]]

Map: 100%|██████████| 1000/1000 [00:00<00:00, 29320.55 examples/s]


In [6]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [7]:
accuracy = evaluate.load("accuracy")
f1 = evaluate.load("f1")

In [8]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return f1.compute(predictions=predictions, references=labels)

In [9]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [10]:
id2label = {0: "NON_VIOLATED", 1: "VIOLATED"}
label2id = {"NON_VIOLATED": 0, "VIOLATED": 1}
model = BigBirdForSequenceClassification.from_pretrained('google/bigbird-roberta-base', num_labels=2, id2label=id2label, label2id=label2id)

/home/leontioso/anaconda3/envs/msc_thesis/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of BigBirdForSequenceClassification were not initialized from the model checkpoint at google/bigbird-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
training_args = TrainingArguments(
    output_dir="../models/bigbird_ecthr_model",
    learning_rate=4e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset= train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [12]:
trainer.train()

RuntimeError: handle_0 INTERNAL ASSERT FAILED at "../c10/cuda/driver_api.cpp":15, please report a bug to PyTorch. 

In [ ]:
task_evaluator = evaluator("text-classification")
results_dict = {}
for metric in ["accuracy", "precision", "recall", "f1"]:
    results = task_evaluator.compute(
        model_or_pipeline="../models/roberta_ecthr_model/checkpoint-1126",
        data=test_dataset,
        metric=metric,
        tokenizer=tokenizer,
        strategy="simple",
        random_state=0,
        input_column='text',
        label_column='labels',
        label_mapping={"NON_VIOLATED": 0.0, "VIOLATED": 1.0},
    )
    metric_name, value = list(results.items())[0]
    results_dict[metric_name] = value

In [ ]:
results_dict